In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import itertools
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics
import re
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import decomposition
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

from math import log
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [ ]:
df = pd.read_csv('Task_1/train/train.csv')
N = len(df)
print(N)
df.head()


stop_words = ["а","е","и","ж","м","о","на","не","ни","об","но","он","мне","мои","мож","она","они","оно","мной","много","многочисленное","многочисленная","многочисленные","многочисленный","мною","мой","мог","могут","можно","может","можхо","мор","моя","моё","мочь","над","нее","оба","нам","нем","нами","ними","мимо","немного","одной","одного","менее","однажды","однако","меня","нему","меньше","ней","наверху","него","ниже","мало","надо","один","одиннадцать","одиннадцатый","назад","наиболее","недавно","миллионов","недалеко","между","низко","меля","нельзя","нибудь","непрерывно","наконец","никогда","никуда","нас","наш","нет","нею","неё","них","мира","наша","наше","наши","ничего","начала","нередко","несколько","обычно","опять","около","мы","ну","нх","от","отовсюду","особенно","нужно","очень","отсюда","в","во","вон","вниз","внизу","вокруг","вот","восемнадцать","восемнадцатый","восемь","восьмой","вверх","вам","вами","важное","важная","важные","важный","вдали","везде","ведь","вас","ваш","ваша","ваше","ваши","впрочем","весь","вдруг","вы","все","второй","всем","всеми","времени","время","всему","всего","всегда","всех","всею","всю","вся","всё","всюду","г","год","говорил","говорит","года","году","где","да","ее","за","из","ли","же","им","до","по","ими","под","иногда","довольно","именно","долго","позже","более","должно","пожалуйста","значит","иметь","больше","пока","ему","имя","пор","пора","потом","потому","после","почему","почти","посреди","ей","два","две","двенадцать","двенадцатый","двадцать","двадцатый","двух","его","дел","или","без","день","занят","занята","занято","заняты","действительно","давно","девятнадцать","девятнадцатый","девять","девятый","даже","алло","жизнь","далеко","близко","здесь","дальше","для","лет","зато","даром","первый","перед","затем","зачем","лишь","десять","десятый","ею","её","их","бы","еще","при","был","про","процентов","против","просто","бывает","бывь","если","люди","была","были","было","будем","будет","будете","будешь","прекрасно","буду","будь","будто","будут","ещё","пятнадцать","пятнадцатый","друго","другое","другой","другие","другая","других","есть","пять","быть","лучше","пятый","к","ком","конечно","кому","кого","когда","которой","которого","которая","которые","который","которых","кем","каждое","каждая","каждые","каждый","кажется","как","какой","какая","кто","кроме","куда","кругом","с","т","у","я","та","те","уж","со","то","том","снова","тому","совсем","того","тогда","тоже","собой","тобой","собою","тобою","сначала","только","уметь","тот","тою","хорошо","хотеть","хочешь","хоть","хотя","свое","свои","твой","своей","своего","своих","свою","твоя","твоё","раз","уже","сам","там","тем","чем","сама","сами","теми","само","рано","самом","самому","самой","самого","семнадцать","семнадцатый","самим","самими","самих","саму","семь","чему","раньше","сейчас","чего","сегодня","себе","тебе","сеаой","человек","разве","теперь","себя","тебя","седьмой","спасибо","слишком","так","такое","такой","такие","также","такая","сих","тех","чаще","четвертый","через","часто","шестой","шестнадцать","шестнадцатый","шесть","четыре","четырнадцать","четырнадцатый","сколько","сказал","сказала","сказать","ту","ты","три","эта","эти","что","это","чтоб","этом","этому","этой","этого","чтобы","этот","стал","туда","этим","этими","рядом","тринадцать","тринадцатый","этих","третий","тут","эту","суть","чуть","тысяч"]

def clean(string):
    string = string.lower()
    letters = 'ёйцукенгшщзхъфывапролджэячсмитьбю qwertyuiopasdfghjklzxcvbnm'
    ans = ''
    string = ''.join(c for c in string if c in letters)
    spliced = string.split(' ')
    string = ''.join((morph.parse(word)[0]).normal_form+' ' for word in spliced if len(word) > 1 and not word in stop_words)
    return string[:-1]

for i in range(N):
    df.iloc[i]['text'] = clean(df.iloc[i]['text'])
    if i%10 == 0:
        print(i)

df.sample(frac = 1, random_state = 42).head()
df.to_csv('clean.csv', sep=';', encoding='utf-8')

In [2]:
clean =  pd.read_csv("clean.csv", sep  = ";")
clean.head()

,Unnamed: 0,author,text
0,0,__label__34,страница бумажный версия номер игорь павло авт...
1,1,__label__50,мелочь проплатить культура конец минувший росс...
2,2,__label__6,юношеский низкий высокий закончить спорый пред...
3,3,__label__12,свет упасть тишина родиться жить москва окончи...
4,4,__label__58,полемика черносотенство прошлое перспектива за...


In [3]:
y = [int(string[9:]) for string in clean.author.values] # remove "__label__" from string and convert to int

xtrain, xvalid, ytrain, yvalid = train_test_split(clean.text.values, y, 
                                                  stratify=clean["author"], 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [4]:
tfv = TfidfVectorizer(min_df=3, analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1) 

tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain)
valild1_tfv = tfv.transform(xvalid)

In [68]:
regr = LogisticRegression(C=1000, penalty = 'l2' )
regr.fit(xtrain_tfv, ytrain)
predictions = regr.predict(valild1_tfv)
accuracy_score(predictions, yvalid)


/Users/Admin/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/Admin/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.9283667621776505

In [ ]:
def pos2nneg(a):
    for i in range(len(a)):
        for j in range(len(a[i])):
            a[i][j] = inf if a[i][j]==0 else -log(a[i][j])
    return a
def hung(a1):
    n, m = len(a1), len(a1[0])#60, 60
    a = np.zeros((n+1,m+1))
    for i in range(n):
        for j in range(m):
            a[i+1][j+1] = a1[i][j]
    #print(a)
    u = [0]*(n+1)
    v = [0]*(m+1)
    p = [0]*(m+1)
    way = [0]*(m+1)
    inf = np.inf
    for i in range(1, n+1):
        p[0] = i
        j0 = 0
        minv = [inf]*(m+1)
        used = [False]*(m+1)
        while True:
            used[j0] = True
            i0 = p[j0]
            delta = inf
            j1 = -1
            for j in range(1, m+1):
                if not used[j]:
                    cur = a[i0][j]-u[i0]-v[j]
                    if cur < minv[j]:
                        minv[j] = cur
                        way[j] = j0
                    if minv[j] < delta:
                        delta = minv[j]
                        j1 = j
            for j in range(m+1):
                if used[j]:
                    u[p[j]] += delta
                    v[j] -= delta
                else:
                    minv[j] -= delta
            j0 = j1
            if p[j0] == 0:
                break
        while True:
            j1 = way[j0]
            p[j0] = p[j1]
            j0 = j1
            if j0 == 0:
                break
    ans = [-1]*(n)
    for j in range(1,m+1):
        ans[p[j]-1] = j-1
    return ans

In [ ]:
for i in range(15,20):
    df = pd.read_csv('Task_1/valid_texts/text_'+str(i)+'.csv')
    df.head()
    df["text"] = df["text"].apply(clean)
    df.to_csv('clean'+str(i)+'.csv', sep=';', encoding='utf-8')

In [35]:
for i in range(15,20):
    validi = pd.read_csv("/Users/Admin/Downloads/clean_valid1/clean"+str(i)+".csv", sep  =';')
    validi_tfv  = tfv.transform(validi["text"])
    predictions = regr.predict_proba(validi_tfv)
    print("author")
    for v in hung(pos2nneg(predictions)):
        print("__label__{}".format(v))

author
__label__48
__label__21
__label__32
__label__44
__label__40
__label__18
__label__29
__label__36
__label__26
__label__56
__label__22
__label__51
__label__11
__label__27
__label__52
__label__58
__label__23
__label__50
__label__38
__label__3
__label__10
__label__12
__label__34
__label__14
__label__46
__label__0
__label__49
__label__41
__label__28
__label__53
__label__24
__label__4
__label__45
__label__7
__label__15
__label__31
__label__42
__label__57
__label__5
__label__2
__label__16
__label__17
__label__43
__label__30
__label__9
__label__47
__label__59
__label__19
__label__37
__label__39
__label__54
__label__55
__label__1
__label__35
__label__6
__label__20
__label__33
__label__8
__label__25
__label__13
author
__label__44
__label__19
__label__57
__label__38
__label__30
__label__4
__label__12
__label__58
__label__51
__label__13
__label__8
__label__47
__label__5
__label__41
__label__24
__label__29
__label__6
__label__49
__label__23
__label__2
__label__53
__label__21
__label__16
__lab